In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/indian-places-to-visit-reviews-data/Review_db.csv


In [15]:
# os.remove("/kaggle/working/doc2vec2.model")

In [7]:
df = pd.read_csv('/kaggle/input/indian-places-to-visit-reviews-data/Review_db.csv');

In [3]:
df.head()

,City,Place,Review,Rating,Name,Date,Raw_Review
0,Aamby Valley City,19 Degree North,aamby valley beautiful place clear blue skies ...,5,Anonymous,NaN,Aamby valley is a beautiful place with its cle...
1,Aamby Valley City,19 Degree North,executed obt akshay thanx team thoroughly enjo...,4,Anonymous,NaN,Very well executed obt by Akshay.... Thanx as ...
2,Aamby Valley City,19 Degree North,awesome experience atv tracts obstacles mainta...,5,Anonymous,NaN,Awesome experience at the ATV\nTracts and obst...
3,Aamby Valley City,19 Degree North,visited aamby valley yesterday short excursion...,4,Anonymous,NaN,we visited the Aamby Valley yesterday for shor...
4,Aamby Valley City,19 Degree North,far mumbai place finest adventure places visit...,5,Anonymous,NaN,"Not far from Mumbai, this place is one of the ..."


In [4]:
df.shape

(1482466, 7)

In [ ]:
df.drop(['Rating', 'Name', 'Date', 'Raw_Review'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.drop_duplicates(subset=['Review'], inplace=True)

In [ ]:
df.shape

In [ ]:
df.to_csv('review_cleaned.csv', index=False)

In [5]:
df = pd.read_csv('/kaggle/working/review_cleaned.csv')

In [3]:
df.head()

,City,Place,Review
0,Aamby Valley City,19 Degree North,aamby valley beautiful place clear blue skies ...
1,Aamby Valley City,19 Degree North,executed obt akshay thanx team thoroughly enjo...
2,Aamby Valley City,19 Degree North,awesome experience atv tracts obstacles mainta...
3,Aamby Valley City,19 Degree North,visited aamby valley yesterday short excursion...
4,Aamby Valley City,19 Degree North,far mumbai place finest adventure places visit...


In [4]:
df.shape

(1464109, 3)

In [ ]:
#df1 = df.head(10000)

In [8]:
!wget http://nlp.stanford.edu/data/glove.6B.zip


--2023-04-18 20:35:28--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-04-18 20:35:29--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-04-18 20:35:29--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [9]:
!unzip glove.6B.zip


Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [2]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors


In [3]:
filename = '/kaggle/working/glove.6B.100d.txt'
# model = KeyedVectors.load_word2vec_format(filename, binary=False)

embeddings_dict = {}
with open(filename, 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [4]:
glove_vectors = embeddings_dict

In [13]:
#glove_vectors = gensim.downloader.load('/kaggle/working/glove-wiki-gigaword-100')

In [5]:
# Define a function to convert a document to a vector using both Doc2Vec and GloVe
def get_doc_vector(model, doc_words, glove_vectors):
    # Get the Doc2Vec vector
    doc_vector = model.infer_vector(doc_words)
    
    # Get the GloVe vector
    glove_vector = np.mean([glove_vectors[w] for w in doc_words if w in glove_vectors], axis=0)
    
    # If it was outside gloVe vocabulary
    try:
        if np.isnan(glove_vector):
            glove_vector = np.zeros(100)
    except ValueError:
        # if glove_vector is not 'nan' ie word exists in vocabulary
        # np.is_nan(glove_vector) will raise ValueError
        ## do nothing here
        pass
    
    # Concatenate the Doc2Vec and GloVe vectors
    return np.concatenate([doc_vector, glove_vector])


In [8]:
df1 = df.sample(n=100000)
df1.shape

(100000, 7)

In [9]:
df1 = df1.reset_index()

In [10]:
df1 = df1.drop('index', axis=1)

In [11]:
df1.head()

,City,Place,Review,Rating,Name,Date,Raw_Review
0,Ahmedabad,Kankaria Lake,visited feel really lucky sharing experience b...,5,Anonymous,NaN,I have been visited there and I feel really lu...
1,Bengaluru,Tipu Sultan Fort and Palace - Bangalore KR Market,tipu sultan summer palace said tipu sultan sum...,4,Anonymous,NaN,The Tipu sultan summer palace is said to be th...
2,New Delhi,Qutub Minar,qutub minar national building built mugal sark...,4,Anonymous,NaN,qutub minar is also a national building but it...
3,Udaipur,Lake Pichola,nice place lovely evening lake fatehsagar better,5,Anonymous,NaN,Nice place for a lovely evening.\nLake fatehsa...
4,Agra,Taj Mahal,unprepared emotional impact view taj silhouett...,5,Anonymous,NaN,I was unprepared for the emotional impact of o...


In [12]:
df1.to_csv('small_db.csv', index=False)

In [13]:
df1 = pd.read_csv('/kaggle/working/small_db.csv')

In [14]:
df1

,City,Place,Review,Rating,Name,Date,Raw_Review
0,Ahmedabad,Kankaria Lake,visited feel really lucky sharing experience b...,5,Anonymous,NaN,I have been visited there and I feel really lu...
1,Bengaluru,Tipu Sultan Fort and Palace - Bangalore KR Market,tipu sultan summer palace said tipu sultan sum...,4,Anonymous,NaN,The Tipu sultan summer palace is said to be th...
2,New Delhi,Qutub Minar,qutub minar national building built mugal sark...,4,Anonymous,NaN,qutub minar is also a national building but it...
3,Udaipur,Lake Pichola,nice place lovely evening lake fatehsagar better,5,Anonymous,NaN,Nice place for a lovely evening.\nLake fatehsa...
4,Agra,Taj Mahal,unprepared emotional impact view taj silhouett...,5,Anonymous,NaN,I was unprepared for the emotional impact of o...
...,...,...,...,...,...,...,...
99995,Kolkata,Howrah Bridge,bridge served main connection howrah calcutta ...,3,Anonymous,NaN,The bridge served as the main connection betwe...
99996,Ahmedabad,Sabarmati Ashram / Mahatma Gandhi's Home,respectfully walked displays explained signifi...,5,Anonymous,NaN,Respectfully walked the displays that explaine...
99997,Chennai,San Thome Church,wanted check church old divine really good old...,5,Anonymous,NaN,I just wanted to check the church ⛪ about it's...
99998,Mumbai,Mount Mary Basilica,chance mumbai th september visit mount mary ch...,5,Anonymous,NaN,If by any chance you are in Mumbai after the 8...


In [15]:
tagged_data = [TaggedDocument(words=desc.split(), tags=[i]) for i, desc in enumerate(df1['Review'])]


KeyboardInterrupt: 

In [ ]:
# Initialize the Doc2Vec model
model = Doc2Vec(vector_size=200, window=10, min_count=5, workers=8, epochs=50)


In [ ]:
# Build the vocabulary of the Doc2Vec model
model.build_vocab(tagged_data)


In [ ]:
# Train the Doc2Vec model
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
# Save the trained Doc2Vec model
model_path = get_tmpfile("doc2vec2.model")
model.save("doc2vec2.model")

In [10]:
model1 = Doc2Vec.load('/kaggle/working/doc2vec2.model')

In [40]:
#model = ''

In [11]:
from nltk import word_tokenize

In [12]:
# Get the document vectors
doc_vectors = [get_doc_vector(model1, word_tokenize(desc), glove_vectors) for desc in df1['Review']]


In [13]:
from nltk.tokenize import word_tokenize


In [15]:
query = 'mountain sunrise view'

In [16]:
tokenized_query = word_tokenize(query)

In [17]:
#tokenized_query

In [18]:
query_vector = get_doc_vector(model1, tokenized_query, glove_vectors)

In [19]:
cosine_similarities = cosine_similarity([query_vector], doc_vectors)


In [20]:
# Get the indices of the most similar documents
most_similar_indices = np.argsort(cosine_similarities[0])[-5:][::-1]


In [21]:
most_similar_indices

array([46479, 18624, 91477, 81297, 25240])

In [22]:
# Print the most similar documents
for i in most_similar_indices:
    print(f"City: {df1.iloc[i]['City']}\nReview: {df1.iloc[i]['Review']}\n")

City: Darjeeling
Review: breathtaking view kanngchenjunga mountain k mountain highest mountain world elevation

City: Panchgani
Review: good view valley horse ride kids flat mountain feature

City: Chitkul
Review: heaven earth place beautiful valley kinnaur visit place

City: Madikeri
Review: view valley good apart sunset sun set hills

City: Ooty
Review: winter clouds clearer day good view mountain

